In [73]:
import pandas as pd
import os

Se carga el archivo

In [74]:
df_production=pd.read_csv('../../data/raw/production_crops_livestock_E_all_data.csv', encoding='latin1')

/var/folders/5k/vqn87b312md3sy0fqhdxhz1c0000gn/T/ipykernel_1858/836067868.py:1: DtypeWarning: Columns (11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59,62,65,68,71,74,77,80,83,86,89,92,95,98,101,104,107,110,113,116,119,122,125,128,131,134,137,140,143,146,149,152,155,158,161,164,167,170,173,176,179,182,185,188,191,194,197) have mixed types. Specify dtype option on import or set low_memory=False.
  df_production=pd.read_csv('../../data/raw/production_crops_livestock_E_all_data.csv', encoding='latin1')


Realizamos una exploración inicial

In [75]:
print("Exploración inicial")
print("Primeras filas:")
print(df_production.head())
print("\nInformación del dataset:")
print(df_production.info())
print("\nEstadísticas descriptivas:")
print(df_production.describe())
print("\nValores nulos por columna:")
print(df_production.isnull().sum())

Exploración inicial
Primeras filas:
   Area Code Area Code (M49)         Area  Item Code Item Code (CPC)  \
0          2            '004  Afghanistan        221          '01371   
1          2            '004  Afghanistan        221          '01371   
2          2            '004  Afghanistan        221          '01371   
3          2            '004  Afghanistan        711          '01654   
4          2            '004  Afghanistan        711          '01654   

                                                Item  Element Code  \
0                                  Almonds, in shell          5312   
1                                  Almonds, in shell          5412   
2                                  Almonds, in shell          5510   
3  Anise, badian, coriander, cumin, caraway, fenn...          5312   
4  Anise, badian, coriander, cumin, caraway, fenn...          5412   

          Element   Unit  Y1961  ... Y2020N    Y2021  Y2021F Y2021N    Y2022  \
0  Area harvested     ha    0.

Verificamos si existen duplicados

In [76]:
print("\nNúmero de duplicados:", df_production.duplicated().sum())
df_production = df_production.drop_duplicates()


Número de duplicados: 0


Definimos las columnas para mantener como identificadores

In [77]:
id_vars = ['Area Code', 'Area', 'Item Code', 'Item', 'Element Code', 'Element', 'Unit']

Identificamos las columnas de años

In [78]:
year_columns = [col for col in df_production.columns if col.startswith('Y')]

Creamos un subconjunto con las columnas deseadas

In [79]:
df_production_subset = df_production[id_vars + year_columns]

In [80]:
print(df_production_subset.columns.to_list())

['Area Code', 'Area', 'Item Code', 'Item', 'Element Code', 'Element', 'Unit', 'Y1961', 'Y1961F', 'Y1961N', 'Y1962', 'Y1962F', 'Y1962N', 'Y1963', 'Y1963F', 'Y1963N', 'Y1964', 'Y1964F', 'Y1964N', 'Y1965', 'Y1965F', 'Y1965N', 'Y1966', 'Y1966F', 'Y1966N', 'Y1967', 'Y1967F', 'Y1967N', 'Y1968', 'Y1968F', 'Y1968N', 'Y1969', 'Y1969F', 'Y1969N', 'Y1970', 'Y1970F', 'Y1970N', 'Y1971', 'Y1971F', 'Y1971N', 'Y1972', 'Y1972F', 'Y1972N', 'Y1973', 'Y1973F', 'Y1973N', 'Y1974', 'Y1974F', 'Y1974N', 'Y1975', 'Y1975F', 'Y1975N', 'Y1976', 'Y1976F', 'Y1976N', 'Y1977', 'Y1977F', 'Y1977N', 'Y1978', 'Y1978F', 'Y1978N', 'Y1979', 'Y1979F', 'Y1979N', 'Y1980', 'Y1980F', 'Y1980N', 'Y1981', 'Y1981F', 'Y1981N', 'Y1982', 'Y1982F', 'Y1982N', 'Y1983', 'Y1983F', 'Y1983N', 'Y1984', 'Y1984F', 'Y1984N', 'Y1985', 'Y1985F', 'Y1985N', 'Y1986', 'Y1986F', 'Y1986N', 'Y1987', 'Y1987F', 'Y1987N', 'Y1988', 'Y1988F', 'Y1988N', 'Y1989', 'Y1989F', 'Y1989N', 'Y1990', 'Y1990F', 'Y1990N', 'Y1991', 'Y1991F', 'Y1991N', 'Y1992', 'Y1992F', 'Y19

Se transforma a formato long

In [81]:
df_long = pd.melt(
    df_production_subset,
    id_vars=id_vars,
    value_vars=year_columns,
    var_name='Year_Flag',
    value_name='Value'
)

Verificamos las columnas de df_long

In [82]:
print(df_long.columns.to_list())

['Area Code', 'Area', 'Item Code', 'Item', 'Element Code', 'Element', 'Unit', 'Year_Flag', 'Value']


Extraemos año y tipo

In [83]:
df_long['Year'] = df_long['Year_Flag'].str.extract(r'(\d{4})').astype(int) 
df_long['Type'] = df_long['Year_Flag'].str.replace(r'Y\d{4}', '', regex=True)
df_long['Type'] = df_long['Type'].replace('', 'Value')

Seleccionamos explicitamente las columnas que usaremos

In [84]:
df_long = df_long[['Area Code', 'Area', 'Item Code', 'Item', 'Element Code', 'Element', 'Unit', 'Year', 'Type', 'Value']]

Verificamos las columnas

In [85]:
print(df_long.columns.to_list())

['Area Code', 'Area', 'Item Code', 'Item', 'Element Code', 'Element', 'Unit', 'Year', 'Type', 'Value']


Pivotamos para reorganizar datos segun el tipo

In [86]:
df_clean = df_long.pivot(
    index=['Area Code', 'Area', 'Item Code', 'Item', 'Element Code', 'Element', 'Unit', 'Year'],
    columns='Type',
    values='Value'
).reset_index()

In [87]:
print(df_clean.columns.to_list())

['Area Code', 'Area', 'Item Code', 'Item', 'Element Code', 'Element', 'Unit', 'Year', 'F', 'N', 'Value']


Renombramos columnas

In [88]:
df_clean.columns = ['Area Code', 'Area', 'Item Code', 'Item', 'Element Code', 'Element', 'Unit', 'Year', 'Flag', 'Note', 'Value']

Estandarizamos formatos de columnas categóricas

In [89]:
df_clean['Area'] = df_clean['Area'].str.strip().str.title()
df_clean['Item'] = df_clean['Item'].str.strip().str.title()
df_clean['Element'] = df_clean['Element'].str.strip().str.title()

Convertimos tipos de datos

In [90]:
df_clean['Value']=pd.to_numeric(df_clean['Value'], errors='coerce')

Manejamos valores faltantes

In [91]:
df_clean=df_clean.dropna(subset=['Value'])

In [92]:
df_clean.to_csv('../../data/processed/production_crops_cleaned.csv', index=False)

Filtramos cultivos de interés

In [93]:
#df_clean=df_clean[df_clean['Item'].isin(["Almonds, in shell"])]

Guardamos el dataset limpio

In [94]:
#df_clean.to_csv('../../data/processed/almonds_production_cleaned.csv', index=False)

Guardamos una opción intermedia de forma opcional

In [95]:
df_long.to_csv('../../data/interim/production_crops_interim.csv', index=False)

Verificamos el resultado final

In [96]:
print("\nPrimeras filas del dataset limpio:")
print(df_clean.head())
print("\nInformación del dataset limpio:")
print(df_clean.info())


Primeras filas del dataset limpio:
    Area Code     Area  Item Code   Item  Element Code         Element Unit  \
31          1  Armenia         15  Wheat          5312  Area Harvested   ha   
32          1  Armenia         15  Wheat          5312  Area Harvested   ha   
33          1  Armenia         15  Wheat          5312  Area Harvested   ha   
34          1  Armenia         15  Wheat          5312  Area Harvested   ha   
35          1  Armenia         15  Wheat          5312  Area Harvested   ha   

    Year Flag               Note    Value  
31  1992    X  Unofficial figure  65500.0  
32  1993    X  Unofficial figure  97900.0  
33  1994    A                NaN  85697.0  
34  1995    A                NaN  60362.0  
35  1996    A                NaN  85450.0  

Información del dataset limpio:
<class 'pandas.core.frame.DataFrame'>
Index: 4047502 entries, 31 to 5021792
Data columns (total 11 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Area Code     int64  
 1